# Data Pre-Processing

In [1]:
import os
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from shapely.geometry import Polygon, Point, MultiPolygon
import math


from zipfile import ZipFile
import numpy as np
import plotly.express as px
import seaborn as sb
import matplotlib.pyplot as plt
import re

os.getcwd()

# Download BRON
#!wget https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/bron/01-01-2010_31-12-2019.zip

'/mnt/c/Users/stijn/OneDrive/Bureaublad/Afstudeerproject/afstudeerproject_repo/scriptie'

In [2]:
!ls

# Locate BRON.zip
!ls -l 01-01-2010_31-12-2019.zip

 01-01-2010_31-12-2019.zip	  Test.ipynb	     introductie.md
 CatBoost_report.xlsx		  _build	     markdown.md
 Data				  _config.yml	     methoden.md
 Data_PreProcessing.ipynb	  _toc.yml	     notebooks.ipynb
 Feature_Engineering.ipynb	  abstract.md	     references.bib
 LogisticRegression_report.xlsx   catboost_info      referentie.md
 Modelling.ipynb		 'chi2(50).xlsx'     requirements.txt
 ROClogistic.png		  images	     verdeling.png
 TOTAALDATA.csv			  inhoudsopgave.md
-rwxrwxrwx 1 smwhering smwhering 186927157 Jul 20  2020 01-01-2010_31-12-2019.zip


## Inladen BRON

In [3]:
# Regex pattern to match names of .txt files in dataset
regex = '([^\/]+(?=\.txt))'

# specifying the zip file name
file_name = "01-01-2010_31-12-2019.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zipf:
    # printing all the contents of the zip file
    df_dct = {}
    
    # Create a dict with al the dataframe .txt tables, the key is the name of this table
    for df in zipf.namelist(): 
        if '.txt' in df and 'Definitie' not in df:
            # Select the name of the .txt data files
            name = re.findall(regex, df)[0] + '_df'
            if name == 'ongevallen_df':
                ongevallen_df = pd.read_csv(zipf.open(df, mode='r'), encoding = 'ISO-8859-1')
                # Drop AVG columns and columns that consist of more then 70% of NaN values
                ongevallen_df.drop(['DATUM_VKL', 'DAG_CODE', 'TIJDSTIP', 'DDL_ID', 'AP4_CODE', 'AP5_CODE', 'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'ANTL_SEH', 'ANTL_GOV', 'ANTL_TDT'], axis=1,  inplace=True)
                print(ongevallen_df.shape)
                # We all years after 2015, because more data has been registered
                threshold = int(len(ongevallen_df) * 0.70)
                ongevallen_df.dropna(axis=1, how='all',thresh=threshold,  inplace=True)
                print(ongevallen_df.shape)
                # Add to df dictionary
                df_dct[name] = ongevallen_df
                ongevallen_df = None
                
            if name == 'partijen_df':
                partijen_df = pd.read_csv(zipf.open(df, mode='r'), encoding = 'ISO-8859-1')
                print(partijen_df.shape)
                threshold = int(len(partijen_df) * 0.70)
                # Drop AVG columns and columns that consist of more then 60% of NaN values
                partijen_df.drop(['NTT_CODE_V', 'GEBDAT', 'GEBJAAR', 'LEEFTIJD', 'NTT_CODE_B', 'BLAASTEST', 'ART8', 'MEDICGEBR', 'TDT_ID_1', 'TDT_ID_2', 'TDT_ID_3'], axis=1,  inplace=True)
                partijen_df.dropna(axis=1, how='all', thresh=threshold, inplace=True)
                print(partijen_df.shape)
                # Add to df dictionary
                df_dct[name] = partijen_df
                partijen_df = None
                
            else:
                data = pd.read_csv(zipf.open(df, mode='r'), encoding = 'ISO-8859-1')
                # Drop 
                data.dropna(axis=1, how='all', inplace=True)
                df_dct[name] = data
                data = None

/home/smwhering/miniconda3/envs/datasc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (6,7,8,29,31,33,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/smwhering/miniconda3/envs/datasc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,24,25,29,31,34,35,39,43,47,52,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1128636, 54)
(1128636, 15)


/home/smwhering/miniconda3/envs/datasc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,24,25,29,31,34,35,39,43,47,52,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/smwhering/miniconda3/envs/datasc/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (29,31,36,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1224079, 41)
(1224079, 4)


## Ongevallen DataFrame

``We selecteren features die van belang zijn bij verkeersongevallen, er zijn namelijk veel features die zorgen voor redundentie in de data. Dit willen we voorkomen om het onderzoek zo transperant en overzichtelijk mogelijk te maken. ``

In [4]:
# Features with id or pair-values that can be used in this research.
filterL = ['VKL_NUMMER', 'JAAR_VKL', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID', 'WSE_ID',
           'WSE_AN', 'BEBKOM','MAXSNELHD', 'WVL_ID', 'WVG_ID', 'WVG_AN','WDK_ID',
           'WDK_AN', 'LGD_ID', 'ZAD_ID', 'WGD_CODE_1', 'WGD_CODE_2', 'BZD_ID_VM1',
           'BZD_VM_AN', 'BZD_ID_IF1', 'BZD_ID_IF2','BZD_ID_TA1', 'BZD_ID_TA2',
           'IND_ALC', 'GME_NAAM', 'PVE_NAAM', 'FK_VELD5']


# Filter Based on 2017
df_total = df_dct['ongevallen_df'].loc[:, (filterL)]
df_total = df_total.merge(df_dct['partijen_df'], on='VKL_NUMMER')
display(df_total.info())

# >= 2017 because the this contains the most amount of data
df_total = df_total[df_total['JAAR_VKL'] >= 2017]
df_total['MAXSNELHD'] = df_total['MAXSNELHD'].astype(str)
df_total.dropna(axis=1, inplace=True, how='all')
display(df_total)
df_total.info()
# display(df_total['MAXSNELHD'])
# display(df_total.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224079 entries, 0 to 1224078
Data columns (total 31 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   VKL_NUMMER  1224079 non-null  int64  
 1   JAAR_VKL    1224079 non-null  int64  
 2   AP3_CODE    1224079 non-null  object 
 3   ANTL_PTJ    1224079 non-null  int64  
 4   AOL_ID      1224079 non-null  int64  
 5   WSE_ID      1088378 non-null  float64
 6   WSE_AN      300 non-null      object 
 7   BEBKOM      927439 non-null   object 
 8   MAXSNELHD   1039808 non-null  float64
 9   WVL_ID      1074245 non-null  float64
 10  WVG_ID      1063267 non-null  float64
 11  WVG_AN      5207 non-null     object 
 12  WDK_ID      1064466 non-null  float64
 13  WDK_AN      2892 non-null     object 
 14  LGD_ID      815109 non-null   float64
 15  ZAD_ID      29 non-null       float64
 16  WGD_CODE_1  1051296 non-null  object 
 17  WGD_CODE_2  793 non-null      object 
 18  BZD_ID_VM1  115087 non

None

,VKL_NUMMER,JAAR_VKL,AP3_CODE,ANTL_PTJ,AOL_ID,WSE_ID,WSE_AN,BEBKOM,MAXSNELHD,WVL_ID,...,BZD_ID_VM1,BZD_ID_IF1,BZD_ID_TA1,IND_ALC,GME_NAAM,PVE_NAAM,FK_VELD5,PTJ_ID,NUMMER,OTE_ID
378640,20170096239,2017,UMS,2,7,5.0,NaN,BI,50.0,2.0,...,NaN,NaN,NaN,NaN,'s-Hertogenbosch,Noord-Brabant,JTE0299223023,27471771,1,1.0
378641,20170096239,2017,UMS,2,7,5.0,NaN,BI,50.0,2.0,...,NaN,NaN,NaN,NaN,'s-Hertogenbosch,Noord-Brabant,JTE0299223023,27471772,2,64.0
378642,20170096245,2017,LET,2,7,5.0,NaN,BI,30.0,2.0,...,NaN,NaN,NaN,NaN,Cuijk,Noord-Brabant,JTE0371230016,27471773,1,1.0
378643,20170096245,2017,LET,2,7,5.0,NaN,BI,30.0,2.0,...,NaN,NaN,NaN,NaN,Cuijk,Noord-Brabant,JTE0371230016,27471774,2,64.0
378644,20170096247,2017,LET,2,6,4.0,NaN,BI,30.0,2.0,...,NaN,NaN,NaN,NaN,Meierijstad,Noord-Brabant,JTE0318206100,27471775,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224074,20190078933,2019,UMS,3,8,1.0,NaN,BI,50.0,2.0,...,NaN,NaN,NaN,NaN,Venlo,Limburg,WVK0600148329,28156351,2,1.0
1224075,20190078933,2019,UMS,3,8,1.0,NaN,BI,50.0,2.0,...,NaN,NaN,NaN,NaN,Venlo,Limburg,WVK0600148329,28156352,3,1.0
1224076,20190078934,2019,LET,3,0,1.0,NaN,BI,30.0,3.0,...,NaN,NaN,NaN,NaN,Beesel,Limburg,WVK0403127004,28156353,1,31.0
1224077,20190078934,2019,LET,3,0,1.0,NaN,BI,30.0,3.0,...,NaN,NaN,NaN,NaN,Beesel,Limburg,WVK0403127004,28156354,2,31.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 566837 entries, 378640 to 1224078
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   VKL_NUMMER  566837 non-null  int64  
 1   JAAR_VKL    566837 non-null  int64  
 2   AP3_CODE    566837 non-null  object 
 3   ANTL_PTJ    566837 non-null  int64  
 4   AOL_ID      566837 non-null  int64  
 5   WSE_ID      509740 non-null  float64
 6   WSE_AN      300 non-null     object 
 7   BEBKOM      516416 non-null  object 
 8   MAXSNELHD   566837 non-null  object 
 9   WVL_ID      508237 non-null  float64
 10  WVG_ID      504457 non-null  float64
 11  WVG_AN      3426 non-null    object 
 12  WDK_ID      513335 non-null  float64
 13  WDK_AN      2063 non-null    object 
 14  LGD_ID      566829 non-null  float64
 15  WGD_CODE_1  506597 non-null  object 
 16  BZD_ID_VM1  40859 non-null   float64
 17  BZD_ID_IF1  25414 non-null   float64
 18  BZD_ID_TA1  5921 non-null    float64
 

In [5]:
# display(df_dct['partijen_df'].info())

## Alternatieven antwoorden
`` Alternatieven antwoorde zijn ingevuld door Politie en/of Rijkswaterstaat waarneer item, met betrekking tot verkeersongelukken nog geen code hebben in het BRON. ``

In [6]:
alternatieve_cols = [col for col in df_total if '_AN' in col]
alternatieve_OMS = df_total[alternatieve_cols]

# Check the alternative answers for road related issues that caused the incident. 
# for col in alternatieve_cols:
#     display(df_total[col].loc[df_total[col].notnull()].unique())

# Gebruik columns zonder alternatieven
cols_zondAlt = df_total.columns.difference(alternatieve_cols)

## Geolocatie pre-processing
``Hier verwerken we de geolocaties tot bruikbare objecttypes om bijvoorbeeld een kaart in plotly te vullen``

In [7]:
# Add locations
geo_df = df_dct['puntlocaties_df'].set_index('FK_VELD5')
df_dct['puntlocaties_df'] = None
geo_df['puntlocatie'] = geo_df[["X_COORD", "Y_COORD"]].apply(Point, axis=1)
geo_df.rename(columns={'X_COORD': "X_COORD_RD", 'Y_COORD': "Y_COORD_RD"}, inplace=True)
geo_df = gpd.GeoDataFrame(geo_df, crs='epsg:28992', geometry=geo_df.puntlocatie).drop(['puntlocatie'], axis=1)
geo_df = geo_df.to_crs('epsg:4326')
geo_df['LONG'] = geo_df.geometry.x
geo_df['LAT'] = geo_df.geometry.y

### Inladen wijkgrenzen van PDOK
``Door de locaties van verkeersincidenten te koppelen aan buurten wijken kunnen we per wijk de relatieve frequency van ongelukken berekenen.``
> ``Bijvoorbeeld: per 'ernst' klasse (matierele schade, gewonden, doden) en voertuig. Deze rel_frequency kan door het model worden gebruikt als numeriek waarde voor het bereken van kansen``

In [8]:
buurten_url = 'https://geodata.nationaalgeoregister.nl/wijkenbuurten2020/wfs?request=GetFeature&service=WFS&version=1.1.0&typeName=wijkenbuurten2020:cbs_buurten_2020&outputFormat=json'
buurtgrenzen = gpd.read_file(buurten_url)
buurtgrenzen = buurtgrenzen.to_crs('epsg:4326')

In [9]:
buurtgrenzen = buurtgrenzen[['buurtnaam',
                             'wijknaam',
                             'geometry']]

In [10]:
# # Using spatial join we can pair FK_VELD5 tot wijken
geo_df_final = gpd.tools.sjoin(geo_df, buurtgrenzen, how="left")
geo_df_final.drop(columns=['index_right'], inplace=True)

In [11]:
geo_df_final

,X_COORD_RD,Y_COORD_RD,geometry,LONG,LAT,buurtnaam,wijknaam
FK_VELD5,,,,,,,
WVK0139255044,69647.746,427626.998,POINT (4.14894 51.83070),4.148937,51.830704,Hooghen Hoeck,Wijk 03 De Struyten
WVK0140207011,70476.791,403619.861,POINT (4.16680 51.61508),4.166801,51.615083,Sint Philipsland,Sint Philipsland
WVK0139135003,69542.500,367942.500,POINT (4.16197 51.29431),4.161970,51.294309,Kern Nieuw Namen,Nieuw Namen
WVK0139135005,69492.201,367951.000,POINT (4.16125 51.29438),4.161247,51.294378,Kern Nieuw Namen,Nieuw Namen
WVK0139135031,69581.733,367863.631,POINT (4.16255 51.29361),4.162551,51.293606,Kern Nieuw Namen,Nieuw Namen
...,...,...,...,...,...,...,...
JTE0474291033,237344.000,445954.000,POINT (6.58622 51.99585),6.586221,51.995851,Nieuwe Kamp,Wijk 01 Lichtenvoorde
JTE0278368041,139094.895,484316.265,POINT (5.15379 52.34653),5.153788,52.346528,Europakwartier,Wijk 04 Almere Poort
JTE0365048030,182700.632,324312.948,POINT (5.78102 50.90791),5.781019,50.907909,Ulestraten,Wijk 01 Ulestraten


## Voeg alles samen tot één DataFrame
- ``Hier word de partijen_df gekoppeld aan het ongevallen_df, hierdoor is is er meer informatie over het voertuig gekoppeld aan de de ongeluk locaties. De sleutels van de dat worden respectievelijk, 'PTJ_ID' en 'VKL_NUMMER'.``
- ``Vervolgens moeten we alle waardes in de columnen opnieuw gaan coderen zodat er een analyses over gedaan kan worden``
- ``Voordat we kunnen hercoderen moeten we eerst alle NaN values omzetten in iets werkbaars, voor string columen zetten NaN om in 'NaN', in numerieke columns transformeren we NaN naar 0``
- ``Vervolgens voegen we de omschrijving toe aan het DataFrame, hierdoor is het gemakkelijk om te zien welke factoren impact hebben met betrekking tot verkeersongelukkenl.``

In [12]:
df_total = df_total[cols_zondAlt]
ID_cols = [col for col in df_total.columns if '_ID' in col]

# Special function for BZD_ID to get the OMS (omschrijving) of categorical value
for col in ID_cols:
    if 'BZD_ID' in col:
        df_total.loc[:,col] = df_total.loc[:,col].map(df_dct['bijzonderheden_df'].set_index('BZD_ID')['BZD_OMS'])
        
# General function for '_ID' columns to get the OMS (omschrijving) of categorical value
for df in df_dct:
    if df != 'ongevallen_df' and df != 'partijen_df' and df != 'puntlocaties_df':
#         print(df)
        cols = set(df_dct[df].columns)
        inters = list(set(ID_cols) & cols)
        if inters:
            # Create OMS (omschrijving) relation to get de specific value
            omschrvng = inters[0][:4]+'OMS'
            # Add tot total dataframe
            df_total.loc[:,inters[0]] = df_total.loc[:,inters[0]].map(df_dct[df].set_index(inters[0])[omschrvng])
            
# Use whole OMS for AP3_CODE and WGD_CODE_1
df_total["AP3_CODE"].replace({"DOD":"Dodelijk", "LET":"Letsel", "UMS":"Uitsluitend materiele schade"}, inplace=True)
df_total["WGD_CODE_1"].replace({"D":"Droog", "R":"Regen", "M":"Mist", "S":"Sneeuw/Hagel", "H":"Harde Windstoten", "O": "Onbekend"}, inplace=True)

# Remove df_dict to clear some memory
df_dct.clear()
df_dct = None
# Reorder de columns and add spatial data tot DF
df_total = df_total[['VKL_NUMMER', 'PTJ_ID', 'ANTL_PTJ', 'AOL_ID', 'AP3_CODE', 'BEBKOM','OTE_ID',
 'IND_ALC','MAXSNELHD','WDK_ID','WGD_CODE_1', 'WSE_ID', 'WVG_ID', 'WVL_ID','LGD_ID', 'GME_NAAM', 'PVE_NAAM', 'FK_VELD5']]

df_TOTAAL = df_total.merge(geo_df_final, on='FK_VELD5')
df_TOTAAL.columns = df_TOTAAL.columns.str.upper()
df_TOTAAL.drop(['FK_VELD5','X_COORD_RD', 'Y_COORD_RD', 'GEOMETRY'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566460 entries, 0 to 566459
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   VKL_NUMMER  566460 non-null  int64  
 1   PTJ_ID      566460 non-null  int64  
 2   ANTL_PTJ    566460 non-null  int64  
 3   AOL_ID      566460 non-null  object 
 4   AP3_CODE    566460 non-null  object 
 5   BEBKOM      516099 non-null  object 
 6   OTE_ID      549269 non-null  object 
 7   IND_ALC     15399 non-null   object 
 8   MAXSNELHD   566460 non-null  object 
 9   WDK_ID      513020 non-null  object 
 10  WGD_CODE_1  506282 non-null  object 
 11  WSE_ID      509428 non-null  object 
 12  WVG_ID      504143 non-null  object 
 13  WVL_ID      507922 non-null  object 
 14  LGD_ID      566452 non-null  object 
 15  GME_NAAM    566460 non-null  object 
 16  PVE_NAAM    566460 non-null  object 
 17  LONG        566460 non-null  float64
 18  LAT         566460 non-null  float64
 19  BU

In [17]:
print(df_TOTAAL.columns)
df_TOTAAL = df_TOTAAL[['PTJ_ID', 'VKL_NUMMER', 'ANTL_PTJ', 'AOL_ID', 'AP3_CODE', 'BEBKOM',
                       'OTE_ID', 'IND_ALC', 'MAXSNELHD', 'WDK_ID', 'WGD_CODE_1', 'WSE_ID',
                       'WVG_ID', 'WVL_ID', 'LGD_ID','LONG', 'LAT', 'BUURTNAAM', 'WIJKNAAM','GME_NAAM', 'PVE_NAAM']]
df_TOTAAL.info()

Index(['PTJ_ID', 'VKL_NUMMER', 'ANTL_PTJ', 'AOL_ID', 'AP3_CODE', 'BEBKOM',
       'OTE_ID', 'IND_ALC', 'MAXSNELHD', 'WDK_ID', 'WGD_CODE_1', 'WSE_ID',
       'WVG_ID', 'WVL_ID', 'LGD_ID', 'GME_NAAM', 'PVE_NAAM', 'LONG', 'LAT',
       'BUURTNAAM', 'WIJKNAAM'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566460 entries, 0 to 566459
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PTJ_ID      566460 non-null  int64  
 1   VKL_NUMMER  566460 non-null  int64  
 2   ANTL_PTJ    566460 non-null  int64  
 3   AOL_ID      566460 non-null  object 
 4   AP3_CODE    566460 non-null  object 
 5   BEBKOM      516099 non-null  object 
 6   OTE_ID      549269 non-null  object 
 7   IND_ALC     15399 non-null   object 
 8   MAXSNELHD   566460 non-null  object 
 9   WDK_ID      513020 non-null  object 
 10  WGD_CODE_1  506282 non-null  object 
 11  WSE_ID      509428 non-null  object 
 12  WVG_ID      504

In [18]:
# df_TOTAAL.to_csv('TOTAALDATA', index_label=['VKL_NUMMER', 'PTJ_ID'])
df_TOTAAL = df_TOTAAL.set_index(['PTJ_ID', 'VKL_NUMMER'])
df_TOTAAL.to_csv('TOTAALDATA.csv')

In [14]:
df_TOTAAL.reset_index(inplace=True)
df_TOTAAL.head()
df_TOTAAL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566460 entries, 0 to 566459
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PTJ_ID      566460 non-null  int64  
 1   VKL_NUMMER  566460 non-null  int64  
 2   ANTL_PTJ    566460 non-null  int64  
 3   AOL_ID      566460 non-null  object 
 4   AP3_CODE    566460 non-null  object 
 5   BEBKOM      516099 non-null  object 
 6   OTE_ID      549269 non-null  object 
 7   IND_ALC     15399 non-null   object 
 8   MAXSNELHD   566460 non-null  object 
 9   WDK_ID      513020 non-null  object 
 10  WGD_CODE_1  506282 non-null  object 
 11  WSE_ID      509428 non-null  object 
 12  WVG_ID      504143 non-null  object 
 13  WVL_ID      507922 non-null  object 
 14  LGD_ID      566452 non-null  object 
 15  GME_NAAM    566460 non-null  object 
 16  PVE_NAAM    566460 non-null  object 
 17  LONG        566460 non-null  float64
 18  LAT         566460 non-null  float64
 19  BU

````{margin}
```{note}
Here is a note!
```
````